<a href="https://colab.research.google.com/github/peppopi/Piratas/blob/main/Image_promps_code_inspector.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@markdown Solo dale clik, no necesita conectarse a google drive, al ejecutar el codigo se te dara la opcion para subir imagenes, puedes subir las  que quieras y a meddia que se suban al entorno de ejecucion las imagenes se iran procesando.
#@markdown
#@markdown El Limite funciona para hacer mas rapido el codigo y no muestra tanta basura como valor de limite pongamos, ajustalo en funcion de la cantidad de tags que necesites ver, entre 500 y 4000, lograras ver una imagen con mas de 40 tags, asi que es exagerar ponerlo muy alto.
import ipywidgets as widgets
from IPython.display import display
from PIL import Image
import io
import re
print('Recuerda que subir la imagen tarda un rato considerable, si la imagen pesa mas un 1mg')
def eliminar_caracteres_invalidos(texto):
    texto_limpio = re.sub(r'[^a-zA-Z0-9,:<> ]', '', texto)
    return texto_limpio

def extraer_primeros_500_bytes(imagen_bytes):
    Limite = 700 #@param {type:"slider", min:500, max:25000, step:100}
    primeros_500_bytes = imagen_bytes[:Limite]  # Limitar a los primeros 500 bytes
    try:
        texto_sin_caracter = eliminar_caracteres_invalidos(primeros_500_bytes.decode('utf-8'))
    except UnicodeDecodeError:
        texto_sin_caracter = eliminar_caracteres_invalidos(primeros_500_bytes.decode('latin-1'))
    return texto_sin_caracter

def cargar_imagen(change):
    archivo = change['new']
    if archivo:
        imagen_bytes = archivo[list(archivo.keys())[0]]['content']
        datos_extraidos = extraer_primeros_500_bytes(imagen_bytes)
        print(datos_extraidos)

# Crear widget de selección de archivo
selector_archivo = widgets.FileUpload()
selector_archivo.observe(cargar_imagen, names='value')

# Mostrar el widget de selección de archivo
display(selector_archivo)
print(' Aca deberias poder ver los tags: Si no ves nada es que tu imagen no guarda tags en su codigo fuente')

Recuerda que subir la imagen tarda un rato considerable, si la imagen pesa mas un 1mg


FileUpload(value={}, description='Upload')

 Aca deberias poder ver los tags: Si no ves nada es que tu imagen no guarda tags en su codigo fuente
JFIFExifIIi,UNICODEsylveon,animal focus,cute,nigth,grass,flowers,  <lora:Sylveon:075>Negative prompt:  EasyNegativeV2, badhands5, badartist,pussySteps: 20, Sampler: DPM 2M Karras, CFG scale: 7, Seed: 2165112820, Size: 512x720, Model: revAnimatedv122, Clip skip: 2, Lora hashes: Sylveon: ab5ca6dd0bde, Version: v1


In [ ]:
#@markdown Los siguientes dos codigos requiern que inicies el Drive y solo pueden procesar imagenes no subidas a internet.
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import re
import time


#@markdown Analiza una sola imagen a la vez, pon el link de tu imagen


ruta_imagen = "/content/drive/MyDrive/Dataset/Copia/00000-revAnimated_v122_3458838191.png"#@param {type:"string"}

#@markdown Ejem: /content/drive/MyDrive/Dataset/Copia/00000-revAnimated_v122_3458838191.png
# Ruta del archivo de imagen
with open(ruta_imagen, "rb") as archivo:
    datos_imagen = archivo.read()

# Decodificar los datos de la imagen en diferentes codecs
codecs_a_probar = ["utf-8", "latin-1", "iso-8859-1"]

texto_imagen = None

for codec in codecs_a_probar:
    try:
        texto_imagen = datos_imagen.decode(codec)
        break
    except UnicodeDecodeError:
        continue

if texto_imagen:
    # Definir el patrón de expresión regular para extraer el fragmento de texto
    patron = r".*?(?<=Xtparameters).*?(?=Clip skip: \d+)"

    # Buscar el fragmento de texto utilizando la expresión regular
    resultado = re.search(patron, texto_imagen, re.DOTALL)

    if resultado:
        fragmento_texto = resultado.group().strip()
    else:
        print()
        fragmento_texto = "No se encontró el fragmento de texto no se encontro : Tu imagen no guarda ningun Tag"
else:
    fragmento_texto = "No se pudo decodificar los datos de la imagen"

# Función para obtener el texto después de "PtEXtparameters" y reemplazar el carácter "�"
def obtener_texto_despues(fragmento_texto):
    texto_despues = ""
    indice_ptex = fragmento_texto.find("PtEXtparameters") + len("PtEXtparameters")
    texto_despues = fragmento_texto[indice_ptex:].replace("�", " Promps:")
    return texto_despues

# Obtener el texto después de "PtEXtparameters" y reemplazar el carácter "�"
texto_despues = obtener_texto_despues(fragmento_texto)

# Imprimir el resultado si el tiempo de procesamiento es menor a 15 segundos
tiempo_inicio = time.time()

if (time.time() - tiempo_inicio) <= 15:
    print(texto_despues)
else:
    print("El procesamiento de la imagen ha excedido los 15 segundos: Tu imagen no tiene tags")

In [ ]:
import os
import re
import time
#@markdown Por carpetas
# Ruta de la carpeta con las imágenes
ruta_carpeta = "/content/drive/MyDrive/Dataset/Copia"#@param {type:"string"}

#@markdown Ejem: /content/drive/MyDrive/Dataset/Copia/
#@markdown
#@markdown Si el inico del Patron no esta en los primeros mil caracteres el codigo considerara que no hay tags, por el momento simpre el texto se pone justo al comienso por lo que es medio una exageracion poner mil caracteres si  queres hacerlo mas rapido bajalos a 100, en ese caso procearas mil imagenes en unos segundos.
archivos = os.listdir(ruta_carpeta)

# Límite de tiempo en segundos
limite_tiempo = 5

# Iterar sobre cada archivo en la carpeta
for archivo in archivos:
    # Verificar si el archivo es una imagen (png o jpg)
    if archivo.endswith(".png") or archivo.endswith(".jpg"):
        # Ruta completa del archivo de imagen
        ruta_imagen = os.path.join(ruta_carpeta, archivo)

        # Leer el archivo de imagen en modo binario
        with open(ruta_imagen, "rb") as archivo_imagen:
            datos_imagen = archivo_imagen.read()

        # Decodificar los datos de la imagen en diferentes codecs
        codecs_a_probar = ["utf-8", "latin-1", "iso-8859-1"]

        texto_imagen = None

        tiempo_inicio = time.time()

        for codec in codecs_a_probar:
            try:
                texto_imagen = datos_imagen.decode(codec)
                break
            except UnicodeDecodeError:
                continue

            # Verificar si ha excedido el límite de tiempo
            if time.time() - tiempo_inicio >= limite_tiempo:
                print("Error: El procesamiento de la imagen ha excedido los 5 segundos.")
                break

        if texto_imagen:
            # Verificar si el inicio del patrón se encuentra en los primeros mil caracteres
            if not texto_imagen[:1000].find("Xtparameters") != -1:
                print("Error: El inicio del patrón no se encuentra en los primeros mil caracteres.")
                continue

            # Definir el patrón de expresión regular para extraer el fragmento de texto
            patron = r".*?(?<=Xtparameters).*?(?=Clip skip: \d+)"

            # Buscar el fragmento de texto utilizando la expresión regular
            resultado = re.search(patron, texto_imagen, re.DOTALL)

            if resultado:
                fragmento_texto = resultado.group().strip()
            else:
                fragmento_texto = "No se encontró el fragmento de texto"
        else:
            fragmento_texto = "No se pudo decodificar los datos de la imagen"

        # Función para obtener el texto después de "PtEXtparameters" y reemplazar el carácter "�"
        def obtener_texto_despues(fragmento_texto):
            texto_despues = ""
            indice_ptex = fragmento_texto.find("PtEXtparameters") + len("PtEXtparameters")
            texto_despues = fragmento_texto[indice_ptex:].replace("�", " Promps:")
            return texto_despues

        # Obtener el texto después de "PtEXtparameters" y reemplazar el carácter "�"
        texto_despues = obtener_texto_despues(fragmento_texto)

        # Imprimir el resultado con el nombre del archivo
        print("Archivo:", archivo)
        print(texto_despues)
        print()
